# FEATURE ENGINEERING FOR TIME SERIES

In [ ]:
import matplotlib.pyplot as plt
import re
import time
#time decorator

def time_estimator(func):
    def wrapper (*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"Elapsed time: {elapsed:.2f} seconds")
        return result
    return wrapper 

def forecast_plot(y_train, y_test, y_pred, target_name):
    
    import seaborn as sns
    last_row_history = pd.Series(y_train[-1])
    last_row_history.index = [y_train.index[-1]]
    
    y_pred_plot = y_pred.append(last_row_history)
    y_pred_plot = y_pred_plot.sort_index()
    
    if y_test.empty==True: 
        sns.set(style ="whitegrid")
        plt.figure(figsize=(11,4))
        plt.title("Forecast results")
        plt.xlabel('Date')
        plt.ylabel(f'{target_name}')
                              
    sns.lineplot(data = y_train, marker = 'o')
    sns.lineplot(data = y_pred_plot, marker = 'o')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.grid(False)
    pit.legend(['History', "Forecast"])
    return None

    y_test_plot = y_test.append(last_row_history)
    y_test_plot = y_test_plot.sort_index()
    
    sns.set(style ="whitegrid")
    plt.figure(figsize=(11,4))
    plt.title("Overall results")
    plt.xlabel('Date')
    plt.ylabel(f'{target_name}')
    sns.lineplot(data = y_train, marker = 'o')
    sns.lineplot(data = y_test_plot, marker = 'o')
    sns.lineplot(data = y_pred_plot, marker = 'o')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.grid(False)
    plt.legend(['History', "Test", "Forecast"])
    
    sns.set(style ="whitegrid")
    plt.figure(figsize=(11,4))
    plt.title("Forecast vs test")
    plt.xlabel('Date')
    plt.ylabel(f'{target_name}')
    sns.lineplot(data = y_test_plot, marker = 'o')
    sns.lineplot(data = y_pred_plot, marker = 'o')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.grid(False)
    plt.legend(["Test", "Forecast"])
    return None 

In [ ]:
equity = eod.filter( (F.col("ric")=="TSLA.0") | (F.col("ric")=="TSLA.0Q") )
equity = equity.toPandas() 
equity = equity.drop_duplicates(subset ='loaddate')
equity = equity.sort_values("loaddate")

In [ ]:
"""
equity database colums:

- loaddate --> Date in format dd/mm/yy
- Volume --> End of day traded Volume
- 1 single equity ticker considered


"""

## Domain based features

In [ ]:
@time estimator
def Feature_engineering_volume(df):
    tsp = df[[ 'week', 'loaddate', 'closeprice', 'openprice', 'volume ']l
    ts = spark. createDataFrame(tsp)
    ts = ts.orderBy(col("loaddate") )
    ts = ts.select("loaddate" , "volume", "closeprice", "openprice")
    ts = ts.withColumn("group" ,F.lit("1") )
    windowSpec = Window.partitionBy("group").orderBy("loaddate")
    ts = ts.withColumn("lag1", F. lag("volume" , 1).over(windowSpec))
    w = Window().partitionBy(F.col('group')).orderBy('loaddate').rowsBetween(-9, 0)
    ts = ts.withColumn("rollingmean", F.avg("volume").over(w))
    
    def computeweights(n):
        random_n = np.random.exponential(scale=1, size=n)
        sum_n = np.sum(random_n)
        scaled = (random_n / sum_n) * 100
        scaled.sort()
        return scaled
    
    def generate_exp_weights(n):
        weights = computeweights(n)
        n=1000
        distribution = computeweights(n)
        plt. plot(distribution)
        return weights 

    weights = generate_exp_weights(10)
    w = Window().partitionBy(F.col('group')).orderBy('loaddate').rowsBetween(-9, 0)

    ts = ts.withColumn('data', sort_array(collect_list(struct('loaddate', 'volume')).over(w))) \ 
            .withColumn('idx', expr("array_position(data, (loaddate, volume)) -1" )) \
            .withColumn('weights', F.array([F.lit(x) for x in weights] )) \
            .withColumn('sum weights', expr("aggregate(weights, 0D, (acc,x) -› acc+x)"))

    ts = ts.withColumn('rollingweightedmean', 
                       expr(""" aggregate( 
                           zip_with(data, weights, (x,y) -> x.volume*y),
                           0D,
                           (acc, x) -› асс+х, 
                           acc -> acc/sum_weights) """))

    ts = ts.drop("data", "idx", "weights", "sum_ weights")
    ts = ts.withColumn('rollingweightedmean', F.coalesce(ts.rollingweightedmean,ts.rollingmean))
    ts = ts.withColumn("sequencetime", row_number().over(Window.orderBy("loaddate")) )
    ts = ts.withColumn("sequencetime_exp", F.col("sequencetime")**2 )

    ts = ts.withColumn("lead1", F.lead("volume").over(windowSpec)) \
            .withColumn("lead2", F.lead("volume", 2).over(windowspec) \
            .withColumn("lead3" , F.lead("volume", 3).over.(windowSpec) \
            .withColumn("lag1" , F.lag("volume" , 1).over(windowSpec) ) \
            .withColumn("lag2", F.lag("volume" ,2).over(windowSpec) ) \
            .withColumn("lag3", F.lag("volume" , 3).over(windowSpec) )

    ts = ts.withColumn( 'lag1' , F.coalesce(ts.lag1, ts.volume))
    ts = ts.withColumn('lag2' ,F.coalesce(ts.lag2,ts.lag1))
    ts = ts.withColumn('lag3', F.coalesce(ts.lag3, ts.lag2))
    ts = ts.withColumn ('lead1', F.coalesce(ts.lead1,ts.volume))
    ts = ts.withColumn ('lead2' F.coalesce(ts.lead2,ts.lead1))
    ts = ts.withColumn ('lead3', F.coalesce(ts.lead3,ts.lead2))

    growing_flag = (F.col("volume") > F.col("lag1")) & (F.col("volume") > F.col("lag2")) \
                        & (F.col("volume") > F.col("lag3"))
    falling_flag = (F.col("volume") > F.col("lead1")) & (F.col("volume") > F.col("lead2")) & 
                        (F.col("volume") > F.col("lead3")) 
    ts = ts.withColumn("growing_flag", F.when(growing_flag, 1). otherwise(0) ) \
            .withColumn ("falling_flag", F.when(falling_flag, 1). otherwise(0) )

    ts = ts.withColumn("changepoint", F.when((F.col("growing_flag")==1) & (F.col("falling_flag")==1) , 1).otherwise(0))
                        \ .drop("growing_flag", "falling_flag")

    ts = ts.withColumn("changelag1", F.lag("changepoint").over(windowSpec)) \
            .withColumn("changelag2", F.lag("changepoint", 2).over(windowSpec) ) \
            .withColumn("changelag3", F.lag("changepoint", 3).over(windowSpec) ) 

    ts = ts.withColumn("changepoint_confirm", F.when( (F.col("changepoint")==1) & (F.col("changelag1")!=1) \
                                                     & (F.col("changelag2")!=1) & (F.col("changelag3")!=1) , 1).otherwise(0) ) 

    ts = ts.withColumn("changepoint_up", F.when((F.co1("changepoint_confirm")==1) & (F.col("volume") > F.col("lag1") ), 1).otherwise(0) ) \
                       .withColumn("changepoint _down", F.when( (F.col("changepoint_confirm")==1) & (F.col("volume")< F.col("lag1") ), 1).otherwise(0) )

    ts = ts.withColumn("trendgroup", F.sum(F.col("changepoint_confirm")).over(Window.orderBy("loaddate")))
    ts = ts.withColumn("trendgroup", F.when(F.col("changepoint_confirm")==1, F.col("trendgroup")-1).otherwise(F.col("trendgroup")))



    windowTrend = Window().partitionBy(F.col('trendgroup')).orderBy('loaddate')
    ts = ts.withColumn("seq_within_trendgroup" ,F.row_number).over(windowTrend))
    changes = ts.select("trendgroup").distinct().rdd.flatMap(lambda x: x).collect()

    trend_expr = [F.when(F.col("trendgroup") == ch, 1).otherwise(0).alias("Trend_n_" + str(ch) ) for ch in changes]

    cols = ts.columns
    for c in trend_expr:
        cols.append(c)

    ts = ts.select(cols)
    trend_list = [col for col in ts.columns if "Trend_n_" in col]

    for trend in trend_list: 
        ts = ts.withColumn(trend+"_lag",F.lag(F.col(trend)).over(windowSpec) ) 
        n = int(trend.split("_")[-1])            
        if n!=0:
            past_col = "Trend_n_" + str(n-1)
            ts = ts.withColumn(trend+"_turnpoint", F.when( (F.col(trend+"_lag")==0) & (F.col(trend)==1) & \ 
                            (F.col(past_col)==0), F.col("loaddate")).otherwise(0))

    turnpoint_list = [col for col in ts.columns if "_turnpoint" in col]
    for turnpoint in turnpoint list:
        turn = ts.select(turnpoint).filter(F.col(turnpoint)!="0").first()[0]
        ts = ts.withColumn(turnpoint, F when(F. col("loaddate") >= turn, 1).otherwise (0) )
        #ts = ts. withCoLumn (turnpoint, F. row_number (). over (Window(). partitionBy(F. col(turnpoint)). orderBy( 'Loaddate')))

    turnpoint_list = [col for col in ts.columns if "_turnpoint" in col]
    for turnpoint in turnpoint_list:

        ts = ts.withColumn(turnpoint + "_2" , F.row_number().over(Window().partitionBy(F.col(turnpoint)).orderBy('loaddate')))
        ts = ts.withColumn(turnpoint + "_2" ,F.when(F.col(turnpoint)==0,0).otherwise(F.col(turnpoint + "_2")））


    to_keep2 = ["sequencetime", "loaddate" , "volume", "rollingweightedmean", "sequencetime_exp", "changepoint_confirm",
                "trendgroup", "seq_within_trendgroup", "closeprice", "openprice"]
    turnpoint_list = [col for col in ts.columns if "_turnpoint_2" in col]
    to_keep2.extend(turnpoint_list)
    ts = ts.select(to_keep2) 

    ts = ts.withColumn("group", F.lit("1") )
    windowSpec = Window.partitionBy("group").orderBy( "loaddate")
    ts = ts.withColumn("lag_closeprice", F.lag("closeprice" ,1).over(windowSpec))
    ts = ts.withColumn("pricechange" ,F.col("closeprice") - F. col("lag_closeprice") )
    ts = ts.withColumn("gain" ,F.when(F.col("pricechange") >=0, F.col("pricechange")).otherwise(0))
    ts = ts.withColumn ("loss", F.when(F.col("pricechange") <0, F.col("pricechange")).otherwise(0) )
    ts = ts.withColumn ("loss", F.when(F.col("loss")!=0, F.col("loss") * (-1)).otherwise(0) )


    ts = ts.orderBy(col("loaddate"))
    window_change_10 = Window.partitionBy("group").orderBy("loaddate").rowsBetween(-9, 0)
    ts = ts withColumn("averagegain", F.avg("gain").over(window_change_10) )
    ts = ts.withColumn ("averageloss" , F.avg("loss").over(window_change_10) )
    ts = ts.orderBy(col("loaddate"))
    ts = ts.withColumn("RS",F.col("averagegain") / F.col("averageloss")) 
    ts = ts.withColumn ("RSI10", 100 - (100 / (1 + F.col("RS")) ) ) 
    ts = ts.withColumn("RSI10", F.when(F.col("averageloss")==0.0, 100).otherwise(F.col("RSI10")) )
    ts = ts.orderBy(col("loaddate"))
    window_change_20ma = Window. partitionBy("group").orderBy("loaddate").rowsBetween(-19, 0)
    ts = ts.withColumn("MA", F.avg("closeprice").over(window_change_20ma))
    ts = ts. withColumn("MA_diff", F.col("MA") - F.col("closeprice"))
    ts = ts. withColumn("expanding_mean_vol", F.avg("volume").over(Window.partitionBy("group").orderBy("loaddate").rowsBetween(Window.unboundedPreceding, 0)))
    ts = ts.withColumn('Day', F.date_format( 'loaddate', 'EEEE'))

    to_keep = ['sequencetime', 'loaddate', 'volume', 'lag', 'rollingweightedmean', ' sequencetime_exp', 'changepoint_confirm', 'trendgroup', 'seq_within_trendgroup', 'closeprice', 'RSI5' , 'RSI10', 'MA diff', 'expanding_mean_vol', 'Day' ]

    turnpoint_list = [col for col in ts.columns if "_turnpoint_2" in col]
    to_keep.extend(turnpoint_list)

    final = ts.toPandas()
    final = final[to_keep3]
    return final

## TSFRESH package implementation

In [ ]:
#!pip install --user tsfresh 
@time_estimator
def feature_tsfresh(final):
    
    from tsfresh import extract_relevant_features 
    from tsfresh import extract features
    from tsfresh.utilities.dataframe_functions import roll_time_series
    import re
    
    temp = final[['loaddate','volume']]
    temp["group"]=1
    df_rolled = roll_time_series(temp, column_id = "group", column_sort ="loaddate")
    df_features = extract_features(df_rolled, column_id = "id", column_sort ="loaddate")
    
    df_features = df_features.reset_index()
    df_features = df_features.fillna(0)
    df_features["loaddate"] = final["loaddate"]
    df_features.drop(["level_0","level_1"],axis=1,inplace=True)
    finaltsfresh = pd.merge(final,df_features, how="left", on="loaddate")
    finaltsfresh= pd.concat([finaltsfresh, pd.get_dummies(final['Day'])] ,axis=1)
    finaltsfresh = finaltsfresh.drop("Day",axis=1)
    
    finaltsfresh = finaltsfresh.rename(columns = lambda x:re.sub('[^A-Za z0-9_]+','',x))
    finaltsfresh = finaltsfresh.loc[:~finaltsfresh.columns.duplicates()].copy()
    
    pw = finaltsfresh.copy(depp=True)
    pw['loaddate3'] = pw['loaddate'].apply(lambda x: pd.to_datetime())
    pw.index = pw.loaddate
    pw=pw.drop('loaddate',axis=1)
    return pw

In [ ]:
final = Feature_engineering_volume(equity)
pw = feature_tsfresh(final)